In [2]:
import pandas as pd

In [3]:
#Load the data

df = pd.read_csv("UK_sol_irrad_0506_RAW.csv",sep=";",skiprows=31)
df.rename(columns={"# Date": "Date"},inplace=True)
df.loc[df.Time=="24:00","Time"]="00:00"
df["Date_merge"] = df[["Date","Time"]].agg(" ".join,axis=1)
df.drop("Date", axis=1, inplace=True)
df.drop("Time", axis=1, inplace=True)
df.rename(columns={"Date_merge": "Date","Global Horiz": "rad"},inplace=True)
df.Date = pd.to_datetime(df.Date, format = "%Y-%m-%d %H:%M")
df.set_index("Date",inplace=True)
df=df[["rad"]]
df.head(25)

,rad
Date,
2005-01-01 01:00:00,0
2005-01-01 02:00:00,0
2005-01-01 03:00:00,0
2005-01-01 04:00:00,0
2005-01-01 05:00:00,0
2005-01-01 06:00:00,0
2005-01-01 07:00:00,0
2005-01-01 08:00:00,0
2005-01-01 09:00:00,7


In [4]:
df.loc[df.rad==-999,:]

,rad
Date,
2005-08-24 07:00:00,-999
2005-08-24 08:00:00,-999
2005-08-24 09:00:00,-999
2005-08-24 10:00:00,-999
2005-08-24 11:00:00,-999
2005-08-24 12:00:00,-999
2005-08-24 13:00:00,-999
2005-08-24 14:00:00,-999
2005-08-24 15:00:00,-999


There are 79 missing values. We'll replace them by the average irrandiance during that hour in that month (without taking into account the -999 values).

In [5]:
df_clean = df[df.rad>=0]
df_grouped = df_clean.groupby([df_clean.index.month,df_clean.index.hour]).agg("mean")
df_grouped.loc[8,"rad"]

Date
0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
5       0.741935
6      29.548387
7     144.200000
8     279.400000
9     395.040000
10    477.760000
11    512.480000
12    535.680000
13    561.920000
14    496.840000
15    453.000000
16    365.640000
17    278.640000
18    147.240000
19     37.225806
20      2.354839
21      0.000000
22      0.000000
23      0.000000
Name: rad, dtype: float64

In [6]:
date_miss = df.loc[df.rad==-999,:].index
for i,d in enumerate(date_miss):
    month = d.month
    hour = d.hour
    df.loc[df.index==d,"rad"]=df_grouped.loc[month,"rad"][hour]

In [7]:
df[df.index.date==pd.to_datetime("20050824").date()]

,rad
Date,
2005-08-24 01:00:00,0.00
2005-08-24 02:00:00,0.00
2005-08-24 03:00:00,0.00
2005-08-24 04:00:00,0.00
2005-08-24 05:00:00,0.00
2005-08-24 06:00:00,0.00
2005-08-24 07:00:00,144.20
2005-08-24 08:00:00,279.40
2005-08-24 09:00:00,395.04


In [27]:
df_cut = df[~(df.index.date == pd.to_datetime("2006-01-01").date())]

In [29]:
df_cut

,rad
Date,
2005-01-01 01:00:00,0.0
2005-01-01 02:00:00,0.0
2005-01-01 03:00:00,0.0
2005-01-01 04:00:00,0.0
2005-01-01 05:00:00,0.0
2005-01-01 06:00:00,0.0
2005-01-01 07:00:00,0.0
2005-01-01 08:00:00,0.0
2005-01-01 09:00:00,7.0


In [30]:
df_cut.to_csv("solar_rad.csv",header=False,index=False)